# Lab 07

## Zadanie 1

Przygotuj demo programu znajdującego miejsca zerowe metodą Newtona. Wykorzystując `argparse` ([link](https://docs.python.org/3.6/library/argparse.html#module-argparse)) lub `optparse` ([link](https://docs.python.org/3.6/library/optparse.html)) obsłuż:
- ustalane punktu startowego,
- wielkość kroku w pochodnej,
- ilość kroków metody,
- dokładność
- pomoc

Program uruchamiamy podając, np.:

`./newton.py x**2+x+1 -h 0.00001`

In [6]:
import argparse
parser = argparse.ArgumentParser(description='Newtons method manager')
parser.add_argument('--func',help='function, use x symbol as variable',type=str, default='x**2 - 1')
parser.add_argument('--start', help='starting point',default=3.0,type=float)
parser.add_argument('--num_steps',help='number of method steps',default=10,type=int)
parser.add_argument('--epsilon',help='early stopping if delta between steps < arg',type=float,default=0.0001)
# parser.add_argument('--help') defaultowo juz dziala wiec chyba nie ma sensu dodawac

# w notebookach trzeba dac pusty string https://medium.com/@data.scientist/ipython-trick-how-to-use-argparse-in-ipython-notebooks-a07423ab31fc
args = parser.parse_args("")
func = lambda x : eval(args.func)
start_point = args.start
num_steps = args.num_steps
eps = args.epsilon

from scipy.optimize import newton
print(newton(func=func,x0=start_point,maxiter=num_steps,rtol=abs(eps)))

1.00000000011666


## Zadanie 2

Zadanie należy wykonać wykorzystując program `BagOfWords` z poprzednich zajęć.


Przerób ją tak, żeby interpunkcja, cyfry i wszelkie inne znaki nie przeszkadzały w parsowaniu tekstu. Uruchom ja na [tekście hamleta](http://www.gutenberg.org/cache/epub/1787/pg1787.txt). Ile razy występuje słowo hamlet? Jak brzmi 10 najczęściej występujących słów?

In [27]:
from collections import defaultdict
import re
class BagOfWords():
    def __init__(self,arg):
        if type(arg).__name__ == 'TextIOWrapper':
            text = arg.read().strip()
        else:
            text = arg
        # text = re.sub('[^0-9a-zA-Z]+', '', text)
        text = text.replace(",","")
        text = text.replace('.','')
        text = text.replace(';','')
        text = text.replace('\n','')
        text = text.replace('-','')
        allwords = text.split(' ')
        self.dict = defaultdict(int)
        for w in allwords:
            self.dict[w] +=1

    def __repr__(self):
        res = ''
        for key, value in self.dict.items():
            res += f'{key} {value},'
        return res
    def __contains__(self, item):
        return item in self.dict

    def __iter__(self):
        sorted_dict = dict(sorted(self.dict.items(),key = lambda item: item[1], reverse=True))
        self.sorted_dict_vals = list(sorted_dict)
# rozwiazanie 2 ponizej razem z 3

## Zadanie 3

Wykorzystując `pickle` zapisz i odczytaj klasy z poprzedniego zadania nakarmonej Hamletem. Porównaj metody i rozmiar.

In [31]:
import pickle
h = BagOfWords(open('hamlet.txt'))
# widzimy ze jest to samo przed i po picklowaniu
print(list(sorted(h.dict.items(),key=lambda item: item[1],reverse=True))[1:11])
pickle.dump(h.dict,open('hamlet.p','wb'))
l = pickle.load(open('hamlet.p','rb'))
# 10 najbardziej popularnych wychodzi tak samo
print(list(sorted(l.items(),key=lambda item: item[1],reverse=True))[1:11])
# ile razy wystepuje slowo hamlet
print(f"Słowo Hamlet występuje {l['Hamlet'] + l['hamlet']} razy")

[('the', 937), ('and', 695), ('of', 652), ('to', 617), ('I', 531), ('you', 469), ('a', 456), ('my', 434), ('in', 387), ('Ham', 358)]
[('the', 937), ('and', 695), ('of', 652), ('to', 617), ('I', 531), ('you', 469), ('a', 456), ('my', 434), ('in', 387), ('Ham', 358)]
Słowo Hamlet występuje 83 razy


## Zadanie 4

Wykorzystując [https://gist.github.com/pamelafox/986163](https://gist.github.com/pamelafox/986163) podaj aktualną godzinę we:
- wszystkich krajach, wyświetlając je zgrupowane względem kontynentów,
- przeprowadź symulacyjne wyświetlanie kolejnych krajów w miarę jak w danym kraju wybija północ, opóżnienie wyświetlania ustaw proporcjonalne do realnego czasu

In [25]:
from countryinfo import countries
from collections import defaultdict
from datetime import datetime
import pytz
# to chyba wypada juz zrobic w pandasach
dict = defaultdict(list)
for el in countries:
    for key, value in el.items():
        dict[key].append(value)
import pandas as pd
df = pd.DataFrame.from_dict(dict)
df['curr_hour'] = df.apply(lambda row : datetime.now(pytz.timezone(row['timezones'][0]) if row['timezones'][0] in list(pytz.all_timezones) else pytz.timezone('Europe/Warsaw')),axis=1)
# teraz polnoc wzgledem Warszawy
timestamp = datetime.now().replace(hour=0,minute=0,second=0)
start_timezone = pytz.timezone('Europe/Warsaw')
localized_timestamp = start_timezone.localize(timestamp)
df['hour_warsaw_midnight'] = df.apply(lambda row : localized_timestamp.astimezone(pytz.timezone(row['timezones'][0])) if row['timezones'][0] in list(pytz.all_timezones) else timestamp,axis=1)

print(df.keys())
for group in df.groupby('continent'):
    print(f"Continent {group[0]}")
    print(group[1][['name','curr_hour']])

Index(['timezones', 'code', 'continent', 'name', 'capital', 'curr_hour',
       'hour_warsaw_midnight'],
      dtype='object')
Continent Africa
                                 name                         curr_hour
5                              Angola  2022-11-22 15:47:48.979383+01:00
13                       Burkina Faso  2022-11-22 14:47:48.979383+00:00
16                            Burundi  2022-11-22 16:47:48.979383+02:00
17                              Benin  2022-11-22 15:47:48.979383+01:00
23                           Botswana  2022-11-22 16:47:48.979383+02:00
27   Democratic Republic of the Congo  2022-11-22 15:47:48.979383+01:00
28              Republic of the Congo  2022-11-22 15:47:48.979383+01:00
29                     CÃ´te d'Ivoire  2022-11-22 14:47:48.979383+00:00
31                           Cameroon  2022-11-22 15:47:48.980433+01:00
36                         Cape Verde  2022-11-22 13:47:48.980433-01:00
40                           Djibouti  2022-11-22 17:47:48.98043

In [24]:
for group in df.groupby('continent'):
    print(f"Continent {group[0]}")
    print(group[1][['name','hour_warsaw_midnight']])

Continent Africa
                                 name              hour_warsaw_midnight
5                              Angola  2022-11-22 00:00:00.855987+01:00
13                       Burkina Faso  2022-11-21 23:00:00.855987+00:00
16                            Burundi  2022-11-22 01:00:00.855987+02:00
17                              Benin  2022-11-22 00:00:00.855987+01:00
23                           Botswana  2022-11-22 01:00:00.855987+02:00
27   Democratic Republic of the Congo  2022-11-22 00:00:00.855987+01:00
28              Republic of the Congo  2022-11-22 00:00:00.855987+01:00
29                     CÃ´te d'Ivoire  2022-11-21 23:00:00.855987+00:00
31                           Cameroon  2022-11-22 00:00:00.855987+01:00
36                         Cape Verde  2022-11-21 22:00:00.855987-01:00
40                           Djibouti  2022-11-22 02:00:00.855987+03:00
46                              Egypt  2022-11-22 01:00:00.855987+02:00
47                            Eritrea  2022-11-

## Zadanie 5

Dla klasy `BagOfWords` napisz metody `save` oraz `load` wykorzystujące `json`'a do zapisu i odczytu danych.

In [3]:
from collections import defaultdict
import json
class BagOfWords():
    def __init__(self,arg):
        if type(arg).__name__ == 'TextIOWrapper':
            text = arg.read().strip()
        else:
            text = arg
        text = text.replace(",","")
        text = text.replace('.','')
        text = text.replace(';','')
        text = text.replace('\n','')
        text = text.replace('-','')
        allwords = text.split(' ')
        self.dict = defaultdict(int)
        for w in allwords:
            self.dict[w] +=1

    def __repr__(self):
        res = ''
        for key, value in self.dict.items():
            res += f'{key} {value},'
        return res
    def __contains__(self, item):
        return item in self.dict

    def __iter__(self):
        sorted_dict = dict(sorted(self.dict.items(),key = lambda item: item[1], reverse=True))
        self.sorted_dict_vals = list(sorted_dict)

    def save(self,filename):
        json_object = json.dumps(self.dict,indent=2)
        with open(filename,"w") as outfile:
            outfile.write(json_object)
    @staticmethod
    def read(filename):
        readfile = open(filename)
        data = json.load(readfile)
        print(data)

b = BagOfWords(open('plik.txt'))
b.save('test.json')
BagOfWords.read('test.json')

{'ala': 2, 'ma': 2, 'kota': 1, 'ele': 1}
